<a href="https://colab.research.google.com/github/zahra-zarrabi/Bot_DeepLearning/blob/master/irani_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyTelegramBotAPI
!pip install retina-face

In [2]:
import telebot
from retinaface import RetinaFace
import tensorflow as tf
import cv2
import numpy as np


In [ ]:
bot = telebot.TeleBot("2031556072:AAHJRL1t2vDV8Jl2y35jSGWIGvIJ_aSuPvY")
model = tf.keras.models.load_model('/content/Model_persion.h5')

def predict(img):
    pred_img = model.predict(img)
    return pred_img

@bot.message_handler(commands=['start'])
def start_handler(message):
    bot.send_message(message.chat.id, ' welcome to irani detector bot. ' + message.from_user.username)
    text = 'please upload image '
    bot.send_message(message.chat.id, text)

@bot.message_handler(content_types=['photo'])
def image(message):
    file_info = bot.get_file(message.photo[len(message.photo) - 1].file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    with open('/content/img.jpg', 'wb') as new_file:
      new_file.write(downloaded_file)
    faces = RetinaFace.extract_faces(img_path='/content/img.jpg', align=True)
    face = faces[0]
    print(face)
    img = cv2.resize(face, dsize=(224, 224))
    img = img / 255
    img = np.expand_dims(img, axis=0)
    lable = predict(img)
    lable = str(np.argmax(lable[0]))
    print(lable)
    if lable=='0':
        bot.send_message(message.chat.id, 'foreign person')
    else:
        bot.reply_to(message,'irani person')

@bot.message_handler(func= lambda message:True)
def send_normal_message(message):
    bot.reply_to(message, 'Not expecting text. ')

bot.polling()